## Linear Regression

In [ ]:
pip install pymc3

In [ ]:
import math
import pymc3 as pm
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

We can generate an example dataset by adding noise to a simple linear relationship.

In [ ]:
N = 10

c = 3
m = 2

x = np.linspace(1, N, N)
y = m * x + c + np.random.normal(size=N)

A plot of y against x looks roughly linear.

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

We might use a normal least-squares regression package to find the best fit line.

In [ ]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression().fit(x.reshape(-1, 1), y)

x_predict = np.linspace(0, 10, 1001)
y_predict = reg.coef_[0] * x_predict + reg.intercept_

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
reg.intercept_

In [ ]:
reg.coef_[0]

We can describe the same model with prograbablistic programming.

In [ ]:
model = pm.Model() 

with model:
    
    m = pm.Normal('m', mu=0.0, sd=1.0)

    c = pm.Normal('c', mu=0.0, sd=1.0)
    
    sd = pm.Exponential('sd', lam=1.0)

    mu = m * x + c
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sd, observed=pm.Data('y', y))

In [ ]:
with model:
    
    trace = pm.sample(draws=2000)

In [ ]:
pm.traceplot(trace, var_names=['m', 'c', 'sd']);

In [ ]:
m = np.mean(trace['m'])
c = np.mean(trace['c'])

x_predict = np.linspace(0, 10, 1001)
y_predict = m * x_predict + c

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
sns.kdeplot(trace['m'], trace['c']);
plt.xlabel('m')
plt.ylabel('c')

In [ ]:
m = trace['m']
c = trace['c']

plt.figure(figsize=(10, 4))
for i in range(1000):
  x_predict = np.linspace(0, 10, 1001)
  y_predict = m[i] * x_predict + c[i]
  plt.plot(x_predict, y_predict, 'r', alpha=0.01);
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

## Handling Outliers


If we have outliers in our dataset, standard linear regression is a bit more tricky.

In [ ]:
y_outlier = y

y_outlier[3] += 20

If we apply linear regression the single outlier affects our estimate of the gradient and intercept.

In [ ]:
reg = LinearRegression().fit(x.reshape(-1, 1), y_outlier)

x_predict = np.linspace(0, 10, 1001)
y_predict = reg.coef_[0] * x_predict + reg.intercept_

plt.figure(figsize=(10, 4))
plt.plot(x, y_outlier, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

However, we can build the existance of outliers into our probablistic model.

In [ ]:
model = pm.Model() 

with model:
    
    m = pm.Normal('m', mu=0.0, sd=1.0)

    c = pm.Normal('c', mu=0.0, sd=1.0)
    
    sd_data = pm.Exponential('sd', lam=1.0)

    sd_outlier = 100

    P_outlier = pm.Bernoulli('outlier', p=0.1, shape = (N))

    sd = pm.math.switch(P_outlier, sd_outlier, sd_data)
    
    mu = m * x + c

    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sd, observed=pm.Data('y', y_outlier))

In [ ]:
with model:
    
    trace = pm.sample(draws=2000)

Taking the mean of the posterior and ploting the resulting straight line shows the effectiveness of this approach.

In [ ]:
m = np.mean(trace['m'])
c = np.mean(trace['c'])

x_predict = np.linspace(0, 10, 1001)
y_predict = m * x_predict + c

plt.figure(figsize=(10, 4))
plt.plot(x, y_outlier, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

We can use the trace to calculate the probability that any point is an outlier.

In [ ]:
plt.bar(x, np.mean(trace['outlier'], 0))
plt.xlabel('x')
plt.ylabel('P_outlier')

## Noise on the X-Axis

Standard linear regression has no way of handling noise on the x-axis.

In [ ]:
N = 10

c = 3
m = 2

x = np.linspace(1, N, N) + np.random.normal(size=N) 
y = m * np.linspace(1, N, N) + c + np.random.normal(size=N)       

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

We can easily incorporate this into our probablistic programming approach.

In [ ]:
model = pm.Model() 

with model:
    
    m = pm.Normal('m', mu=0.0, sd=1.0)

    c = pm.Normal('c', mu=0.0, sd=1.0)
    
    sd = pm.Exponential('sd', lam=1.0)

    mu_x = pm.Uniform('mu_x', 0, 20, shape=(N))

    X_obs = pm.Normal('X_obs', mu=mu_x, sd=sd, observed=pm.Data('x', x))

    mu_y = m * X_obs + c

    Y_obs = pm.Normal('Y_obs', mu=mu_y, sd=sd, observed=pm.Data('y', y))

In [ ]:
with model:
    
    trace = pm.sample(draws=2000)

In [ ]:
pm.traceplot(trace, var_names=['m', 'c', 'sd']);

In [ ]:
m = np.mean(trace['m'])
c = np.mean(trace['c'])

x_predict = np.linspace(0, 10, 1001)
y_predict = m * x_predict + c

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
m = trace['m']
c = trace['c']

plt.figure(figsize=(10, 4))
for i in range(1000):
  x_predict = np.linspace(-2, 12, 1001)
  y_predict = m[i] * x_predict + c[i]
  plt.plot(x_predict, y_predict, 'r', alpha=0.01);
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

## Linear regression with a maximum value

We can also imagine any combination of piece-wise functions. A common example would be a sensor that reads linearly and then stops.

In [ ]:
N = 10

maxY = 15

c = 3
m = 2

x = np.linspace(1, N, N)
y = np.minimum(m * x + c, maxY) + np.random.normal(size=N)

In [ ]:
plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.xlabel('x')
plt.ylabel('y');

In [ ]:
model = pm.Model() 

with model:
    
    m = pm.Normal('m', mu=0.0, sd=1.0)

    c = pm.Normal('c', mu=0.0, sd=1.0)
    
    sd = pm.Exponential('sd', lam=1.0)

    maxY = pm.Exponential('maxY', lam=1.0)

    mu = pm.math.minimum(m * x + c, maxY)
    
    Y_obs = pm.Normal('Y_obs', mu=mu, sd=sd, observed=pm.Data('y', y))

In [ ]:
with model:
    
    trace = pm.sample(draws=2000)

In [ ]:
pm.traceplot(trace, var_names=['m', 'c', 'sd', 'maxY']);

In [ ]:
m = np.mean(trace['m'])
c = np.mean(trace['c'])
maxY = np.mean(trace['maxY'])

x_predict = np.linspace(0, 10, 1001)
y_predict = np.minimum(m * x_predict + c, maxY)

plt.figure(figsize=(10, 4))
plt.plot(x, y, 'o');
plt.plot(x_predict, y_predict, 'r');
plt.xlabel('x')
plt.ylabel('y');